In [1]:
#import packages
import numpy as np
from numpy import loadtxt
import pylab as pl
from IPython import display
#from RcTorch import *
from rctorchprivate import *
from matplotlib import pyplot as plt
from scipy.integrate import odeint
import time
import matplotlib.gridspec as gridspec

#this method will ensure that the notebook can use multiprocessing (train multiple 
#RC's in parallel) on jupyterhub or any other linux based system.
try:
    mp.set_start_method("spawn")
except:
    pass
torch.set_default_tensor_type(torch.FloatTensor)
%matplotlib inline
start_time = time.time()

In [2]:
#torch.Relu()

In [3]:
lineW = 3
lineBoxW=2
plt.rcParams['text.usetex'] = True

### This notebook demonstrates how to use RcTorch to find optimal hyper-paramters for the differential equation $\dot y + q(t) y = f(t) $.

Simple population:  <font color='blue'>$\dot y + y =0$  </font>
* Analytical solution: <font color='green'>$y = y_0 e^{-t}$</font>

In [4]:
#define a reparameterization function, empirically we find that g= 1-e^(-t) works well)
def reparam(t, order = 1):
    
    exp_t = torch.exp(-t)
    derivatives_of_g = []
    
    g = 1 - exp_t
    g_dot = 1 - g
    return g, g_dot

In [5]:
def plot_predictions(RC, results, integrator_model, ax = None):
    """plots a RC prediction and integrator model prediction for comparison
    Parameters
    ----------
    RC: RcTorchPrivate.esn
        the RcTorch echostate network to evaluate. This model should already have been fit.
    results: dictionary
        the dictionary of results returned by the RC after fitting
    integrator model: function
        the model to be passed to odeint which is a gold standard integrator numerical method
        for solving ODE's written in Fortran. You may find the documentation here:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
    ax: matplotlib.axes._subplots.AxesSubplot
        If provided, the function will plot on this subplot axes
    """
    X = RC.X.cpu()
    if not ax:
        fig, ax = plt.subplots(1,1, figsize = (6,6))
    for i, y in enumerate(results["ys"]):
        y = y.cpu()
        if not i:
            labels = ["RC", "Integrator Solution"]
        else:
            labels = [None, None]
        ax.plot(X, y, color = "dodgerblue", label = labels[0], linewidth = lineW + 1, alpha = 0.9)

        #calculate the integrator prediction:
        int_sol = odeint(integrator_model, y0s[i], np.array(X.cpu().squeeze()))
        int_sol = torch.tensor(int_sol)
        
        #plot the integrator prediction
        ax.plot(X, int_sol, '--', color = "red", alpha = 0.9, label = labels[1],  linewidth = lineW)
    
    plt.ylabel(r'$y(t)$');
    ax.legend();
    ax.tick_params(labelbottom=False)
    plt.tight_layout()

def covert_ode_coefs(t, ode_coefs):
    """ converts coefficients from the string 't**n' or 't^n' where n is any float
    Parameters
    ----------
    t: torch.tensor
        input time tensor
    ode_coefs: list
        list of associated floats. List items can either be (int/floats) or ('t**n'/'t^n')
    Returns
    -------
    ode_coefs
    """
    type_t = type(t)
    for i, coef in enumerate(ode_coefs):
        if type(coef) == str:
            if coef[0] == "t" and (coef[1] == "*" or (coef[1] == "*" and coef[2] == "*")):
                pow_ = float(re.sub("[^0-9.-]+", "", coef))
                ode_coefs[i]  = t ** pow_
                print("alterning ode_coefs")
        elif type(coef) in [float, int, type_t]:
            pass
        else:
            assert False, "ode_coefs must be a list floats or strings of the form 't^pow', where pow is a real number."
    return ode_coefs
    

def plot_rmsr(RC, results, force, ax = None):
    """plots the residuals of a RC prediction directly from the loss function
    Parameters
    ----------
    RC: RcTorchPrivate.esn
        the RcTorch echostate network to evaluate. This model should already have been fit.
    results: dictionary
        the dictionary of results returned by the RC after fitting
    force: function
        the force function describing the force term in the population equation
    ax: matplotlib.axes._subplots.AxesSubplot
        If provided, the function will plot on this subplot axes
    """
    if not ax:
        fig, ax = plt.subplots(1,1, figsize = (10, 4))
    X = RC.X.cpu()
    ys, ydots = results["ys"], results["ydots"]
    
    residuals = []
    force_t = force(X)
    for i, y in enumerate(ys):
        ydot = ydots[i]
        y = y.cpu()
        ydot = ydot.cpu()
        
        ode_coefs = covert_ode_coefs(t = X, ode_coefs = RC.ode_coefs)
        
        resids = custom_loss(X, y, ydot, None, 
                             force_t = force_t, 
                             ode_coefs = RC.ode_coefs,
                             mean = False)
        if not i:
            resids_tensor = resids
            label = r'{Individual Trajectory RMSR}'
        else:
            resids_tensor = torch.cat((resids_tensor, resids), axis = 1)
            label = None
        resids_specific_rmsr = torch.sqrt(resids/1) 
            
        ax.plot(X, resids_specific_rmsr, color = "orangered", alpha = 0.4, label = label, linewidth = lineW-1)
        residuals.append(resids)
    
    mean_resid = torch.mean(resids_tensor, axis =1)
    rmsr = torch.sqrt(mean_resid)
    ax.plot(X, rmsr, 
               color = "blue", 
               alpha = 0.9, 
               label = r'{RMSR}',
               linewidth = lineW-0.5)

    ax.legend(prop={"size":16});
    
    ax.set_xlabel(r'$t$')
    ax.set_yscale("log")
    ax.set_ylabel(r'{RMSR}')

## task 1: cross check burn in for all three experiments (burn in should be embedded into hps)


In [6]:
def driven_force(X, A = 1):
    """ a force function, specifically f(t) = sin(t)
    Parameters
    ----------
    X: torch.tensor
        the input time tensor
    
    Returns
    -------
    the force, a torch.tensor of equal dimension to the input time tensor.
    """
    return A*torch.sin(X)

def no_force(X):
    """ a force function (returns 0)
    
    Parameters
    ----------
    X: torch.tensor
        the input time tensor
    
    Returns
    -------
    the force, in this case 0.
    """
    return 0

lam =1
def custom_loss(X , y, ydot, out_weights, lam = lam, force_t = None, reg = False, 
               ode_coefs = None, init_conds = None, 
                enet_alpha = None, enet_strength =None, mean = True):
    """ The loss function of the ODE (in this case the population equation loss)
    Parameters
    ----------
    X: torch.tensor
        The input (in the case of ODEs this is time t)
    y: torch.tensor
        The response variable
    ydot: torch.tensor
        The time derivative of the response variable
    enet_strength: float
        the magnitude of the elastic net regularization parameter. In this case there is no e-net regularization
    enet_alpha: float
        the proportion of the loss that is L2 regularization (ridge). 1-alpha is the L1 proportion (lasso).
    ode_coefs: list
        this list represents the ODE coefficients. They can be numbers or t**n where n is some real number.
    force: function
        this function needs to take the input time tensor and return a new tensor f(t)
    reg: bool
        if applicable (not in the case below) this will toggle the elastic net regularization on and off
    reparam: function
        a reparameterization function which needs to take in the time tensor and return g and gdot, which 
        is the reparameterized time function that satisfies the initial conditions.
    init_conds: list
        the initial conditions of the ODE.
    mean: bool
        if true return the cost (0 dimensional float tensor) else return the residuals (1 dimensional tensor)
        
    Returns
    -------
    the residuals or the cost depending on the mean argument (see above)
    """
    #with paramization
    L =  ydot  + lam * y - force_t
    
#     if reg:
#         #assert False
#         weight_size_sq = torch.mean(torch.square(out_weights))
#         weight_size_L1 = torch.mean(torch.abs(out_weights))
#         L_reg = enet_strength*(enet_alpha * weight_size_sq + (1- enet_alpha) * weight_size_L1)
#         L = L + 0.1 * L_reg 
    
    L = torch.square(L)
    if mean:
        L = torch.mean(L)
    return L

In [7]:
np.log10(0.671015444528619)

-0.1732674837112427

In [8]:
#declare the initial conditions (each initial condition corresponds to a different curve)
y0s = np.arange(-5.1, 5.1, 1)#2.1
len(y0s)

bounds_dict = {"connectivity" : (-2, -0.1), #log space
               "spectral_radius" : (0.9, 1.0), #lin space
               "n_nodes" : (50, 52.5), 
               "regularization" : (-3.5, 3.5), #log space
               "leaking_rate" : (0.5, 0.95),    #linear space
               "input_connectivity" : (0.1, 1.0),
               "dt" : -2, #log space
               #"input_scaling": (0,1),
               "bias": (-0.75,0.75), #linear space
#                "mu" : 0.0, #(-0.5, 0.5),
#                "sigma" : (-3, 0.5)
               }

#set up data
x0, xf = 0, 5
nsteps = int(abs(xf - x0)/(10**bounds_dict["dt"]))
xtrain = torch.linspace(x0, xf, nsteps, requires_grad=False).view(-1,1)
int(xtrain.shape[0] * 0.5)

250

In [9]:
# common cv arguments:
cv_declaration_args = {"interactive" : False, 
                       "n_jobs" : 10, #batch size is parallel
                       "cv_samples" : 1, #2 number of cv_samples, random start points
                       "initial_samples" : 50, # 50 number of random samples before optimization starts
                       "validate_fraction" : 0.01, #validation prop of tr+val sets
                       "log_score" : True, #log-residuals
                       "random_seed" : 209, # random seed
                       "ODE_order" : 1, #order of eq
                       "subsequence_prop" : 0.8,
                       #"solve_sample_prop" : 0.8,
                       "n_outputs" : 1,
                       "n_inputs" : 1,
                       "length_min" : 2 ** (-11),#2 **(-7), 
                       "success_tolerance" : 3}

In [10]:
t2_hps = {'dt': 0.01,
 'connectivity': 0.5196913030864784,
 'spectral_radius': 1.2,#0.9868528246879578,
 'n_nodes': 302.3799133300781,
 'regularization': 8.465829646562826,
 'leaking_rate': 0.9, #0.5895532369613647,
 'input_connectivity': 0.42385581135749817,
 'bias': 0.32629919052124023}

log_vars = ['connectivity', 'llambda', 'llambda2', 'enet_strength',
                         'noise', 'regularization', 'dt', 'gamma_cyclic', 'sigma',
                         #'input_connectivity', 'feedback_connectivity'
                         ]
tmp = {}
for key, val in t2_hps.items():
    if key in log_vars:
        tmp[key] = np.log10(val)
    else:
        tmp[key] = val
tmp

{'dt': -2.0,
 'connectivity': -0.2842545509338379,
 'spectral_radius': 1.2,
 'n_nodes': 302.3799133300781,
 'regularization': 0.9276695251464844,
 'leaking_rate': 0.9,
 'input_connectivity': 0.42385581135749817,
 'bias': 0.32629919052124023}

In [11]:
bounds_dict = {"connectivity" : (-0.4, -0.1), #log space
               "spectral_radius" : (1.1, 1.3), #lin space
               "n_nodes" : (20, 22.5), 
               "regularization" : (-3, 3.0), #log space
               "leaking_rate" : (0.85, 0.95),    #linear space
               "input_connectivity" : (0.2, 0.8),
               "dt" : -1, #log space
               #"input_scaling": (0,1),
               "bias": (-0.5, 0.5), #linear space
#                "mu" : 0.0, #(-0.5, 0.5),
#                "sigma" : (-3, 0.5)
               }

In [ ]:
#%%time
#declare the esn_cv optimizer: this class will run bayesian optimization to optimize the bounds dict.
#for more information see the github.
esn_cv = EchoStateNetworkCV(bounds = bounds_dict,
                            esn_burn_in = 500, #states to throw away before calculating output
                            #combine len of tr + val sets
                            **cv_declaration_args,
                            reservoir_weight_dist = "uniform",
                            turbo_batch_size = 1
                            )
#optimize the network:
opt = True
if opt:
    t2_pop_hps = esn_cv.optimize(x = xtrain,
                              reparam_f = reparam, 
                              ODE_criterion = custom_loss,
                              init_conditions = [y0s], 
                              force = driven_force,
                              n_trust_regions = 8,
                              max_evals = 1500,
                              tr_score_prop = 0.99,
                              ode_coefs = ["t^2", 1],
                              reg_type = "driven_pop")

FEEDBACK: None , device: None
cpu
Model initialization and exploration run...


2021-09-11 02:11:23,054	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


In [ ]:
t2_pop_hps 

In [ ]:
esn_cv.recover_hps()

In [ ]:
esn_cv.X_turbo.shape, esn_cv.Y_turbo.shape

In [ ]:
esn_cv.recover_hps()

In [ ]:
plt.plot(esn_cv.Y_turbo)

In [ ]:
t2_pop_hps

### Simple population

In [ ]:
#@ray.remote(num_gps = 1)

In [ ]:
import ray
hi = ray.remote

class conditional_ray(object):
    def __init__(self):
        cuda_is_available = torch.cuda.is_available()
        if cuda_is_available:
            self.decorator = ray.remote(num_gps = 1)
        else:
            self.decorator = ray.remote
    def __call__(self, func):
        self.decorator(func)

In [ ]:
# common cv arguments:
cv_declaration_args = {"interactive" : True, 
                       "n_jobs" :8, #batch size is parallel
                       "cv_samples" : 1, #2 number of cv_samples, random start points
                       "initial_samples" : 10, # 50 number of random samples before optimization starts
                       "validate_fraction" : 0.3, #validation prop of tr+val sets
                       "log_score" : True, #log-residuals
                       "random_seed" : 209, # random seed
                       "ODE_order" : 1, #order of eq
                       "subsequence_prop" : 0.8,
                       #see turbo ref:
                       "n_outputs" : 1,
                       "n_inputs" : 1,
                       "length_min" : 2 ** (-7),#2 **(-7), 
                       "success_tolerance" : 3}

In [ ]:
#declare the bounds dict. We search for the variables within the specified bounds.
# if a variable is declared as a float or integer like n_nodes or dt, these variables are fixed.
bounds_dict = {"connectivity" : (-2.2, -0.12), #log space
               "spectral_radius" : (1, 10), #lin space
               "n_nodes" : 250, 
               "regularization" : (-4, 4), #log space
               "leaking_rate" : (0, 1),    #linear space
               "dt" : -2.5, #log space
               "bias": (-0.75,0.75), #linear space
               "mu" : 0.0,#(-0.5, 0.5),
               "sigma" : (-3, 0.5)
               }



In [ ]:
%%time
#declare the esn_cv optimizer: this class will run bayesian optimization to optimize the bounds dict.
#for more information see the github.
esn_cv = EchoStateNetworkCV(bounds = bounds_dict,
                            esn_burn_in = 500, #states to throw away before calculating output
                            **cv_declaration_args,
                            feedback = False,
                            reservoir_weight_dist = "normal"
                            )
#optimize the network:
opt = True
if opt:
    simple_pop_hps = esn_cv.optimize(x = xtrain,
                              n_trust_regions = 4,
                              max_evals = 200,
                              reparam_f = reparam, 
                              ODE_criterion = custom_loss,
                              init_conditions = [y0s], 
                              force = no_force,
                              ode_coefs = [1, 1],
                              #n_outputs = 1,
                              reg_type = "simple_pop")

In [ ]:
plt.plot(esn_cv.X_turbo[:,0])

In [ ]:
def descale_hps(self, denorm = True):
   
    best_vals = self.X_turbo
    scores = esn_cv.Y_turbo
    
    if denorm:
        #denormed_ = self.denormalize_bounds(best_vals)
        denormed_ = self.denormalize_bounds(best_vals)
    else:
        denormed_ = best_vals
#         try:
#             denormed_ = denormalize_bounds(best_vals)
#         except:
#             print("FAIL")

    #best_vals = X_turbo[torch.argmax(Y_turbo)]
    all_hps =[]
    
    for i, row in enumerate(denormed_):
        #####Bad temporary code to change it back into a dictionaryf
        denormed_free_parameters = list(zip(self.free_parameters, row))
        denormed_free_parameters = dict([ (item[0], item[1].item()) for item in denormed_free_parameters])

        hps = denormed_free_parameters
        for fixed_parameter in self.fixed_parameters:
            hps = {fixed_parameter : self.bounds[fixed_parameter], **hps }

        #log_vars = ['connectivity', 'llambda', 'llambda2', 'noise', 'regularization', 'dt']
        for var in self.log_vars:
            if var in hps:
                hps[var] = 10. ** hps[var] 
        hps["score"] = float(scores[i])
        all_hps.append(hps)


    # Return best parameters
    return pd.DataFrame(all_hps)

    

In [ ]:
import seaborn as sns
plt.scatter(df['connectivity'], df["score"])

In [ ]:
#plt.plot(esn_cv._errorz[0])
#esn_cv.Y_turbo

In [ ]:
simple_pop_hpss = {'dt': 0.0031622776601683794,
 'n_nodes': 250,
 'connectivity': 0.035335003384821,
 'spectral_radius': 6.823965072631836,
 'regularization': 0.0004151860381238963,
 'leaking_rate': 0.22981758415699005,
 'bias': 0.3198728561401367}

feedback_hps = {'dt': 0.0031622776601683794,
 'n_nodes': 250,
 'connectivity': 0.042459767294240155,
 'spectral_radius': 6.535794734954834,
 'regularization': 0.002854726449745577,
 'leaking_rate': 0.20386245846748352,
 'bias': -0.5434672832489014}


normal_reservoir_hps = {'mu': 0.0,
                         'dt': 0.0031622776601683794,
                         'n_nodes': 500,
                         'connectivity': 0.5553696929611397,
                         'spectral_radius': 6.276016712188721,
                         'regularization': 6.4142668843564135,
                         'leaking_rate': 0.8516905903816223,
                         'bias': -0.11652106046676636,
                         'sigma': 1.1307638250980523}

In [ ]:
%%time
pop_RC = EchoStateNetwork(**normal_reservoir_hps,
                          random_state = 209, 
                          dtype = torch.float32,
                          reservoir_weight_dist = "normal",
                          activation_function = "sigmoid",
                          n_inputs = 1,
                          n_outputs = 1)

train_args = {"X" : xtrain.view(-1,1),        
              "burn_in" : 500, 
              "ODE_order" : 1,   
              "force" : no_force, 
              "reparam_f" : reparam,
              "ode_coefs" : [1, 1]}


pop_results = pop_RC.fit(init_conditions = [y0s,1],
                        SOLVE = True,
                        train_score = True, 
                        ODE_criterion = custom_loss,
                         n_outputs = 1,
                        **train_args)

In [ ]:
def simple_pop(y, t, t_pow = 0, force_k = 0, k = 1):
    dydt = -k * y *t**t_pow + force_k*np.sin(t)
    return dydt

In [ ]:
#TODO: show results outside BO range

In [ ]:
# some particularly good runs:

# simple_pop_hps = {'dt': 0.0031622776601683794,
#  'n_nodes': 250,
#  'connectivity': 0.13615401772200952,
#  'spectral_radius': 4.1387834548950195,
#  'regularization': 0.00028325262824591835,
#  'leaking_rate': 0.2962796092033386,
#  'bias': -0.5639935731887817}

# opt_hps = {'dt': 0.0031622776601683794,
#  'n_nodes': 250,
#  'connectivity': 0.7170604557008349,
#  'spectral_radius': 1.5755887031555176,
#  'regularization': 0.00034441529823729916,
#  'leaking_rate': 0.9272222518920898,
#  'bias': 0.1780446171760559}

# opt_hps = {'dt': 0.0017782794100389228,
#  'n_nodes': 250,
#  'connectivity': 0.11197846061157432,
#  'spectral_radius': 1.7452095746994019,
#  'regularization': 0.00012929296298723957,
#  'leaking_rate': 0.7733328938484192,
#  'bias': 0.1652531623840332}

In [ ]:
np.log10(0.05)

In [ ]:

fig = plt.figure(figsize = (9, 7)); gs1 = gridspec.GridSpec(3, 3);
ax = plt.subplot(gs1[:-1, :])

gts = plot_predictions(RC = pop_RC, 
                       results = pop_results, 
                       integrator_model = simple_pop, 
                       ax = ax)

ax = plt.subplot(gs1[-1, :])
plot_data = plot_rmsr(pop_RC, 
                      results = pop_results, 
                      force = no_force, 
                      ax = ax)

In [ ]:
import numpy as np
np.split()

In [ ]:
best_idx = np.argmin(pop_results["scores"])
plt.plot(pop_results['ydots'][best_idx])

### Driven population:

In [ ]:
#declare the bounds dict. We search for the variables within the specified bounds.
# if a variable is declared as a float or integer like n_nodes or dt, these variables are fixed.
bounds_dict = {"connectivity" : (-2, -0.12), #log space
               "spectral_radius" : (1, 10), #lin space
               "n_nodes" : (250,253), 
               "regularization" : (-4, 4), #log space
               "leaking_rate" : (0, 1),    #linear space
               "dt" : -2.5, #log space
               "bias": (-0.75,0.75), #linear space,
               "mu" : 0.0, #(-0.5, 0.5),
               "sigma" : (-3, 0.5)
               }

In [ ]:
%%time
#declare the esn_cv optimizer: this class will run bayesian optimization to optimize the bounds dict.
#for more information see the github.
esn_cv = EchoStateNetworkCV(bounds = bounds_dict,
                            feedback = True,
                            esn_burn_in = 500, #states to throw away before calculating output
                            #combine len of tr + val sets
                            **cv_declaration_args,
                            activation_function = 'sigmoid',
                            reservoir_weight_dist = "normal"
                            )
#optimize the network:
opt = True
if opt:
    driven_pop_hps = esn_cv.optimize(x = xtrain,
                              reparam_f = reparam, 
                              ODE_criterion = custom_loss,
                              init_conditions = [y0s], 
                              force = driven_force,
                              ode_coefs = [1, 1],
                              max_evals = 200,
                              n_trust_regions = 5,
                              reg_type = "driven_pop")

In [ ]:
esn_cv

In [ ]:
#y0s = np.arange(-10, 10.1, 1)
len(y0s)

In [ ]:
%%time
driven_RC = EchoStateNetwork(**driven_pop_hps,#driven_pop_hps,
                             random_state = 209, 
                             dtype = torch.float32,
                             reservoir_weight_dist = 'normal',
                             activation_function = 'sigmoid',
                             n_inputs = 1,
                             n_outputs = 1)

train_args = {"X" : xtrain.view(-1,1),        
              "burn_in" : 500, 
              "ODE_order" : 1,   
              "force" : driven_force, 
              "reparam_f" : reparam,
              "ode_coefs" : [1, 1]}


driven_results = driven_RC.fit(init_conditions = [y0s,1],
                    SOLVE = True,
                    train_score = True, 
                    ODE_criterion = custom_loss,
                    n_outputs = 1,
                    **train_args)

In [ ]:
def driven_pop(y, t, t_pow = 0, force_k = 1, k = 1):
    dydt = -k * y *t**t_pow + force_k*np.sin(t)
    return dydt


In [ ]:
#driven_pop_hps

In [ ]:
fig = plt.figure(figsize = (9, 7)); gs1 = gridspec.GridSpec(3, 3);
ax = plt.subplot(gs1[:-1, :])

gts = plot_predictions(RC = driven_RC, 
                       results = driven_results, 
                       integrator_model = driven_pop, 
                       ax = ax)

ax = plt.subplot(gs1[-1, :])
plot_data = plot_rmsr(driven_RC, 
                      results = driven_results, 
                      force = driven_force, 
                      ax = ax)

#### Driven t^2 Population:

In [ ]:
#declare the initial conditions (each initial condition corresponds to a different curve)
#y0s = np.arange(-10, 10.1, 0.1)
len(y0s)

In [ ]:
torch.log(torch.exp(torch.tensor(1, dtype = torch.float32)))

In [ ]:
np.log10(0.005)

In [ ]:
#declare the bounds dict. We search for the variables within the specified bounds.
# if a variable is declared as a float or integer like n_nodes or dt, these variables are fixed.

t2_hps =  {'n_nodes': 50,
           'connectivity': 0.09905712745750006,
           'spectral_radius': 1.8904799222946167,
           'regularization': 714.156090350679,
           'leaking_rate': 0.031645022332668304,
           'bias': -0.24167031049728394,
           'dt' : 0.005}

bounds_dict = {"connectivity" : (-1, -0.1), #log space
               "spectral_radius" : (1.0, 3.0), #lin space
               "n_nodes" : (250, 253), 
               "regularization" : (-3.5, 3.5), #log space
               "leaking_rate" : (0.5, 0.95),    #linear space
               "dt" : -2, #log space
               "input_scaling": 0.0,
               "bias": (-0.75,0.75), #linear space
#                "mu" : 0.0, #(-0.5, 0.5),
#                "sigma" : (-3, 0.5)
               }

In [ ]:
%%time
#declare the esn_cv optimizer: this class will run bayesian optimization to optimize the bounds dict.
#for more information see the github.
esn_cv = EchoStateNetworkCV(bounds = bounds_dict,
                            esn_burn_in = 300, #states to throw away before calculating output
                            #combine len of tr + val sets
                            **cv_declaration_args,
                            #reservoir_weight_dist = "uniform"
                            )
#optimize the network:
opt = True
if opt:
    t2_pop_hps = esn_cv.optimize(x = xtrain,
                              reparam_f = reparam, 
                              ODE_criterion = custom_loss,
                              init_conditions = [y0s], 
                              force = driven_force,
                              n_trust_regions = 4,
                              max_evals = 450,
                              
                              ode_coefs = ["t^2", 1],
                              reg_type = "driven_pop")

In [ ]:
esn_cv.recover_hps()

In [ ]:

def recover_hps(self, k):
#     print(self.X_turbo[torch.argmax(self.Y_turbo)])
#     print(self.Y_turbo.shape)
    ys = self.Y_turbo.long() 
    topk = torch.topk(ys, k, dim = 0)
    print(topk.indices)
    topk = topk.indices
    best_vals = self.X_turbo[topk, :]
    
    
    best_Xs = []
    for i in range(best_vals.shape[0]):
        best_Xs.append(best_vals[i].view(-1,))
    best_hps = []
    for best_vals in best_Xs:
        print(best_vals)
        denormed_ = self.denormalize_bounds(best_vals)

#         try:
#             denormed_ = denormalize_bounds(best_vals)
#         except:
#             print("FAIL")

        #best_vals = X_turbo[torch.argmax(Y_turbo)]

        #####Bad temporary code to change it back into a dictionaryf
        denormed_free_parameters = list(zip(self.free_parameters, denormed_))
        denormed_free_parameters = dict([ (item[0], item[1].item()) for item in denormed_free_parameters])

        best_hyper_parameters = denormed_free_parameters
        for fixed_parameter in self.fixed_parameters:
            best_hyper_parameters = {fixed_parameter : self.bounds[fixed_parameter], **best_hyper_parameters }

        #log_vars = ['connectivity', 'llambda', 'llambda2', 'noise', 'regularization', 'dt']
        for var in self.log_vars:
            if var in best_hyper_parameters:
                best_hyper_parameters[var] = 10. ** best_hyper_parameters[var] 
        best_hps.append(dict(best_hyper_parameters))

    # Return best parameters
    return best_hps

In [ ]:
hps = recover_hps(esn_cv, 4)
hps

In [ ]:
hps

In [ ]:
#solution run:
# t2_hps =  {'n_nodes': 500,
#            'connectivity': 0.09905712745750006,
#            'spectral_radius': 1.8904799222946167,
#            'regularization': 714.156090350679,
#            'leaking_rate': 0.031645022332668304,
#            'bias': -0.24167031049728394,
#            'dt' : 0.005}

In [ ]:
def t2_pop(y, t, t_pow = 2, force_k = 1, k = 1):
    dydt = -k * y *t**t_pow + force_k*np.sin(t)
    return dydt

t2_large_pop_hps = {'dt': 0.0031622776601683794,
  'n_nodes': 5000,
  'connectivity': 0.011375554517426858,
  'spectral_radius': 1.001483678817749,
  'regularization': 0.10551396568378141,
  'leaking_rate': 0.6042274236679077,
  'bias': -0.5456951856613159}

t2_large_pop_hps = {'dt': 0.0031622776601683794,
  'n_nodes': 5000,
  'connectivity': 0.006769240463316872,
  'spectral_radius': 1.02934992313385,
  'regularization': 2.553968845569174,
  'leaking_rate': 0.5708888173103333,
  'bias': 0.11009591817855835}

t2_hps =  {'n_nodes': 500,
           'connectivity': 0.09905712745750006,
           'spectral_radius': 1.8904799222946167,
           'regularization': 714.156090350679,
           'leaking_rate': 0.031645022332668304,
           'bias': -0.24167031049728394,
           'dt' : 0.005,
          'mu' : None}

t2_hps = {'dt': 0.01,
 'connectivity': 0.01941342977956559,
 'spectral_radius': 0.9240224361419678,
 'n_nodes': 100.79619598388672,
 'regularization': 119.62205010230673,
 'leaking_rate': 0.5126264691352844,
 'bias': -0.6684032678604126}

t2_hps = {'dt': 0.01,
 'connectivity': 0.011935769253512066,
 'spectral_radius': 0.9255421757698059,
 'n_nodes': 100.69910430908203,
 'regularization': 0.0025961285365239143,
 'leaking_rate': 0.5886231064796448,
 'bias': -0.11111664772033691,
 }
t2_hps = {'dt': 0.01,
 'connectivity': 0.5196913030864784,
 'spectral_radius': 1.2,#0.9868528246879578,
 'n_nodes': 302.3799133300781,
 'regularization': 8.465829646562826,
 'leaking_rate': 0.9, #0.5895532369613647,
 'input_connectivity': 0.42385581135749817,
 'bias': 0.32629919052124023}

# t2_normal_reservoir_hps = {'mu': 0.0,
#  'dt': 0.0031622776601683794,
#  'connectivity': 0.046269292530735924,
#  'spectral_radius': 2.9632248878479004,
#  'n_nodes': 252.12603759765625,
#  'regularization': 1629.0326850836211,
#  'leaking_rate': 0.03967997804284096,
#  'bias': -0.017130792140960693,
#  'sigma': 0.004813828205433559}
y0s = np.arange(-5.1, 5.1, 1)
len(y0s)

In [ ]:
%%time
t2_RC = EchoStateNetwork(**t2_hps,
                         #reservoir_weight_dist = 'normal',
                         #activation_function = "tanh",
                         random_state = 209, 
                         dtype = torch.float32,
                         n_inputs = 1)

train_args = {"X" : xtrain.view(-1,1),        
              "burn_in" : 300, 
              "ODE_order" : 1,   
              "force" : driven_force, 
              "reparam_f" : reparam,
              "ode_coefs" : ["t^2", 1],
              }

t2_results = t2_RC.fit(init_conditions = [y0s,1],
                        
                        n_outputs = 1,
                        SOLVE = True,
                        train_score = True, 
                        ODE_criterion = custom_loss,
                        **train_args)

In [ ]:
fig = plt.figure(figsize = (9, 7)); gs1 = gridspec.GridSpec(3, 3);
ax = plt.subplot(gs1[:-1, :])

gts = plot_predictions(RC = t2_RC, 
                       results = t2_results, 
                       integrator_model = t2_pop, 
                       ax = ax)

ax = plt.subplot(gs1[-1, :])
plot_data = plot_rmsr(t2_RC, 
                      results = t2_results, 
                      force = driven_force, 
                      ax = ax)

In [ ]:
end_time = time.time()
print(f'Total notebook runtime: {end_time - start_time:.2f} seconds')